In [30]:
import sys
!{sys.executable} -m pip install mysql-connector-python
!{sys.executable} -m pip install nba_api
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install sqlalchemy




  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/68/89/247a6a5bfcc5f2f27b08ce7a1fbc2ee9e54fa09ccc9fa48e8c0e08e78cbd/SQLAlchemy-2.0.21-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.2.0 from https://files.pythonhosted.org/packages/24/21/7d397a4b7934ff4028987914ac1044d3b7d52712f30e2ac7a2ae5bc86dd0/typing_extensions-4.8.0-py3-none-any.whl.metadata
  Using cached typing_extensions-4.8.0-py3-none-any.whl.metadata (3.0 kB)
     ---------------------------------------- 0.0/192.5 kB ? eta -:--:--
     -- ------------------------------------- 10.2/192.5 kB ? eta -:--:--
     ------------- ----------------------- 71.7/192.5 kB 975.2 kB/s eta 0:00:01
     -------------------------------------- 192.5/192.5 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.0 MB 25.1 MB/s eta 0:00:01
   ---------------------- ------

In [34]:
from dotenv import load_dotenv
import os
load_dotenv()
import mysql.connector as sql
import pandas as pd
from sqlalchemy import create_engine

host=os.getenv("DB_HOST")
database=os.getenv("DB_NAME")
user=os.getenv("DB_USERNAME")
password=os.getenv("DB_PASSWORD")


conn = sql.connect(host=os.getenv("DB_HOST"),
                   database=os.getenv("DB_NAME"),
                   user=os.getenv("DB_USERNAME"),
                   password=os.getenv("DB_PASSWORD"),
                    )
mycursor = conn.cursor()
# Create the SQLAlchemy engine using the MySQL connection
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

In [32]:
mycursor.execute("drop view vw_last_game")
mycursor.execute("""CREATE VIEW vw_last_game as 
            select game_id
                  ,game_date
                  ,city
                  ,nickname
                  ,team_id

            from
                (select sgs.*
                       ,row_number() over (partition by team_id order by game_date,load_date desc) as rn
                from stg_game_stats sgs) max_dates
            where max_dates.rn = 1""") 

In [57]:
query = """ 
            select game_id
                  ,game_date
                  ,city
                  ,nickname
                  ,team_id

            from
                (select sgs.*
                       ,row_number() over (partition by team_id order by game_date desc) as rn
                from stg_game_stats sgs) max_dates
            where max_dates.rn = 1"""
pd.read_sql_query(query,engine).sort_values('game_id').game_id.values

array(['22300082', '22300100', '22300101', '22300103', '22300106',
       '22300107', '22300114', '22300116', '22300117', '22300118',
       '22300119', '22300120', '22300121', '22300126', '22300127',
       '22300130', '22300131', '22300132', '22300133', '22300134',
       '22300135', '22300136', '22300137', '22300138', '22300139',
       '22300140', '22300141', '22300142', '22300143', '22300144'],
      dtype=object)

In [50]:
query = """ select wf.away_team_id as team_id
                  ,wf.game_id
                  ,wf.game_date
                  ,wf.season
                  ,wf.away_last_game_oe as last_game_oe
                  ,wf.away_last_game_home_win_pctg as last_game_home_win_pctg
                  ,wf.away_rest_day_cnt as num_rest_days
                  ,wf.away_last_game_away_win_pctg as last_game_away_win_pctg
                  ,wf.AWAY_LAST_GAME_SEASON_WIN_PCTG as last_game_total_win_pctg
                  ,wf.away_last_game_rolling_scoring_margin as last_game_rolling_scoring_margin
                  ,wf.away_last_game_rolling_oe as last_game_rolling_oe
            from wrk_wl_features wf
            join vw_last_game lg on wf.away_team_id = lg.team_id and wf.game_id = lg.game_id

            union

            select wf.home_team_id as team_id
                  ,wf.game_id
                  ,wf.game_date
                  ,wf.season
                  ,wf.home_last_game_oe as last_game_oe
                  ,wf.home_last_game_home_win_pctg as last_game_home_win_pctg
                  ,wf.home_rest_day_cnt as num_rest_days
                  ,wf.home_last_game_away_win_pctg as last_game_away_win_pctg
                  ,wf.home_LAST_GAME_SEASON_WIN_PCTG as last_game_total_win_pctg
                  ,wf.home_last_game_rolling_scoring_margin as last_game_rolling_scoring_margin
                  ,wf.home_last_game_rolling_oe as last_game_rolling_oe
            from wrk_wl_features wf
            join vw_last_game lg on wf.home_team_id = lg.team_id and wf.game_id = lg.game_id

"""

In [55]:
#
query = """select * from vw_wl_features"""
pd.read_sql_query(query,engine)

,team_id,game_id,game_date,season,last_game_oe,last_game_home_win_pctg,rest_day_cnt,last_game_away_win_pctg,last_game_season_win_pctg,last_game_rolling_scoring_margin,last_game_rolling_oe
0,1610612737,22300175,2023-11-11,2023-24,0.521368,0.428571,2,0.562500,0.555556,-8.000000,0.521368
1,1610612738,22300188,2023-11-13,2023-24,0.620968,0.900000,2,0.562500,0.782609,23.000000,0.620968
2,1610612739,22300191,2023-11-13,2023-24,0.550459,0.230769,2,0.666667,0.478261,2.666667,0.559063
3,1610612740,22300183,2023-11-12,2023-24,0.609375,0.375000,0,0.461538,0.461538,-14.333333,0.600343
4,1610612741,22300190,2023-11-13,2023-24,0.543860,0.470588,0,0.200000,0.360656,2.333333,0.580490
5,1610612742,22300183,2023-11-12,2023-24,0.546218,0.750000,4,0.900000,0.900000,-2.333333,0.547037
6,1610612743,22300182,2023-11-12,2023-24,0.580357,0.933333,4,0.600000,0.869565,8.000000,0.610823
7,1610612744,22300186,2023-11-12,2023-24,0.520661,0.333333,0,0.631579,0.576923,-6.333333,0.531234
8,1610612745,22300182,2023-11-12,2023-24,0.526316,0.722222,0,0.000000,0.565217,11.000000,0.575645
9,1610612746,22300179,2023-11-12,2023-24,0.529412,0.692308,4,0.000000,0.391304,-4.000000,0.529412


In [53]:
mycursor.execute("drop view vw_last_game")
mycursor.execute("""CREATE VIEW vw_last_game as 
            select game_id
                  ,game_date
                  ,city
                  ,nickname
                  ,team_id

            from
                (select sgs.*
                       ,row_number() over (partition by team_id order by load_date desc,game_date desc) as rn
                from stg_game_stats sgs) max_dates
            where max_dates.rn = 1""") 

In [27]:
from datetime import datetime 
datetime.today().strftime("%Y-%m-%d")

'2023-11-12'

In [26]:
query = f"""update stg_game_stats
set load_date = {datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")} ; commit;  """ # '22300144'""" where id = 1610612757

mycursor.execute(f'{query}')

DatabaseError: 1105 (HY000): syntax error at position 52 near '01'

In [29]:

query = """ select * from wrk_wl_features """
df = pd.read_sql_query(query,engine).dropna()
x = df[df['SEASON'].isin(['2020-21','2021-22'])][['HOME_LAST_GAME_OE', 'HOME_LAST_GAME_HOME_WIN_PCTG',
       'HOME_REST_DAY_CNT', 'HOME_LAST_GAME_AWAY_WIN_PCTG',
       'HOME_LAST_GAME_SEASON_WIN_PCTG',
       'HOME_LAST_GAME_ROLLING_SCORING_MARGIN', 'HOME_LAST_GAME_ROLLING_OE',
       'AWAY_LAST_GAME_OE', 'AWAY_LAST_GAME_HOME_WIN_PCTG',
       'AWAY_REST_DAY_CNT', 'AWAY_LAST_GAME_AWAY_WIN_PCTG',
       'AWAY_LAST_GAME_SEASON_WIN_PCTG',
       'AWAY_LAST_GAME_ROLLING_SCORING_MARGIN', 'AWAY_LAST_GAME_ROLLING_OE']]
y = df[df['SEASON'].isin(['2020-21','2021-22'])]['HOME_W']

In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import preprocessing 
from sklearn.metrics import classification_report
X =x 

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.33)

# Standard Scaling Prediction Variables
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
scaled_data_train = scaler.transform(X_train)

scaler.fit(X_test)
scaled_data_test = scaler.transform(X_test)

#Logistic Regression

model = LogisticRegression()
model.fit(scaled_data_train,y_train)
model.score(scaled_data_test,y_test)

0.6401062416998672

In [21]:
import numpy as np 
y_pred = model.predict_proba(scaled_data_test)
np.vectorize(lambda x: f'{x * 100:.2f}%')(y_pred[:,1])
#print(classification_report(y_test,y_pred))

array(['63.12%', '43.21%', '60.42%', '28.47%', '28.24%', '52.23%',
       '37.00%', '50.28%', '27.46%', '70.91%', '62.96%', '63.06%',
       '55.96%', '28.96%', '42.82%', '55.18%', '56.11%', '46.92%',
       '48.55%', '79.26%', '72.85%', '69.13%', '40.35%', '52.61%',
       '78.74%', '64.56%', '54.83%', '37.85%', '42.32%', '23.49%',
       '80.73%', '57.66%', '66.56%', '64.16%', '65.76%', '43.63%',
       '42.77%', '71.09%', '40.67%', '64.50%', '53.95%', '29.32%',
       '41.83%', '53.84%', '58.28%', '68.86%', '64.46%', '51.57%',
       '40.01%', '52.81%', '50.79%', '71.97%', '60.11%', '48.83%',
       '61.39%', '45.34%', '69.86%', '27.46%', '74.68%', '64.69%',
       '75.80%', '29.41%', '70.30%', '60.21%', '38.47%', '46.75%',
       '49.07%', '64.59%', '68.14%', '40.20%', '41.32%', '66.07%',
       '71.24%', '40.98%', '68.34%', '66.98%', '67.10%', '27.48%',
       '67.50%', '57.74%', '65.24%', '66.78%', '52.44%', '35.09%',
       '74.28%', '44.50%', '33.15%', '34.56%', '29.83%', '48.7

In [54]:
import pickle 
model_string = pickle.dumps(model).hex()
data_dict = {'model_name':['NBA Home Team Win Loss'],'version':['v1'],'create_dt':['2023-11-05'],'model_object_hex':[model_string]}
data = pd.DataFrame(data_dict)
data['create_dt'] = pd.to_datetime(data['create_dt'])
data.to_sql('model_object', engine, if_exists='append', index=False, method='multi')

1

SyntaxError: can't use starred expression here (3058542084.py, line 1)

In [56]:
query = "select * from model_object where version = 'v2'"
data = pd.read_sql_query(query,engine)
model_hex = data.loc[data.model_name == 'NBA Home Team Win Loss']['model_object_hex'][0]
pickle.loads(bytes.fromhex(model_hex)).score(scaled_data_test,y_test)



0.6108897742363878

In [286]:
query = """ select cast(gameId as signed) as GAME_ID
                  ,w_home as home_win_flag
             from 
                stg_nba_schedule sns
             join
                stg_game_stats sgs on sns.gameId = sgs.game_id """
pd.read_sql_query(query,engine)



,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,TEAM_TURNOVERS,...,BLK,PTS,AVG_REB,AVG_PTS,DQ,OFFENSIVE_EFFICIENCY,SCORING_MARGIN,SEASON,GAME_DATE,GAME_ID
0,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,8,124,63.0,124.0,0,0.653226,29.0,2020-21,2021-05-16,22001066
1,OPPONENTS,Rockets,1610612745,0,1,0,0,0,1,1,...,5,95,51.0,95.0,0,0.459016,-29.0,2020-21,2021-05-16,22001066
2,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,4,116,58.0,116.0,0,0.555556,23.0,2020-21,2021-05-13,22001049
3,OPPONENTS,Magic,1610612753,0,1,0,0,0,1,0,...,6,93,62.0,93.0,0,0.517241,-23.0,2020-21,2021-05-13,22001049
4,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,5,120,56.0,120.0,0,0.573913,4.0,2020-21,2021-05-12,22001042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7075,OPPONENTS,Pistons,1610612765,0,1,0,0,0,1,0,...,3,106,50.0,106.0,0,0.550459,-11.0,2022-23,2023-02-27,22200919
7076,Detroit,Pistons,1610612765,1,0,1,0,0,0,1,...,3,118,60.0,118.0,0,0.572581,6.0,2022-23,2023-02-03,22200785
7077,OPPONENTS,Hornets,1610612766,0,1,0,0,0,1,1,...,2,112,46.0,112.0,1,0.525424,-6.0,2022-23,2023-02-03,22200785
7078,Charlotte,Hornets,1610612766,0,1,0,1,0,0,0,...,6,134,65.0,134.0,1,0.552846,-7.0,2022-23,2022-12-14,22200414


In [2]:
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam, gamerotation
import pandas as pd
import numpy as np
import json
import difflib
import time
import requests

# Retry Wrapper 
def retry(func, retries=3):
    def retry_wrapper(*args, **kwargs):
        attempts = 0
        while attempts < retries:
            try:
                return func(*args, **kwargs)
            except requests.exceptions.RequestException as e:
                print(e)
                time.sleep(30)
                attempts += 1

    return retry_wrapper

# Get Single Game aggregation columns

def getSingleGameMetrics(gameID,homeTeamID,awayTeamID,awayTeamNickname,seasonYear,gameDate):

    @retry
    def getGameStats(teamID,gameID,seasonYear):
        #print(f'game_id: {gameID} team_id: {teamID} season:{seasonYear}')
        gameStats = cumestatsteam.CumeStatsTeam(game_ids='00'+str(gameID),league_id ="00",
                                               season=seasonYear,season_type_all_star="Regular Season",
                                               team_id = teamID).get_normalized_json()

        #print(gameStats)
        gameStats = pd.DataFrame(json.loads(gameStats)['TotalTeamStats'])

        return gameStats

    data = getGameStats(homeTeamID,gameID,seasonYear)
    #print(data)
    data.at[1,'NICKNAME'] = awayTeamNickname
    data.at[1,'TEAM_ID'] = awayTeamID
    data.at[1,'OFFENSIVE_EFFICIENCY'] = (data.at[1,'FG'] + data.at[1,'AST'])/(data.at[1,'FGA'] - data.at[1,'OFF_REB'] + data.at[1,'AST'] + data.at[1,'TOTAL_TURNOVERS'])
    data.at[1,'SCORING_MARGIN'] = data.at[1,'PTS'] - data.at[0,'PTS']

    data.at[0,'OFFENSIVE_EFFICIENCY'] = (data.at[0,'FG'] + data.at[0,'AST'])/(data.at[0,'FGA'] - data.at[0,'OFF_REB'] + data.at[0,'AST'] + data.at[0,'TOTAL_TURNOVERS'])
    data.at[0,'SCORING_MARGIN'] = data.at[0,'PTS'] - data.at[1,'PTS']

    data['SEASON'] = seasonYear
    data['GAME_DATE'] = gameDate
    data['GAME_ID'] = gameID

    return data

In [7]:
sql_query = """ SELECT 
                    sss.* 
                FROM 
                    stg_season_schedule sss
                LEFT JOIN 
                    stg_game_stats sgs1 on sss.game_id = sgs1.game_id and sss.home_team_id = sgs1.team_id
                LEFT JOIN 
                    stg_game_stats sgs2 on sss.game_id = sgs2.game_id and sss.away_team_id = sgs2.team_id
                WHERE 
                    sgs1.team_id is null
                      """

# Execute the SQL query and return the result as a DataFrame
df = pd.read_sql_query(sql_query, engine)
df

,MATCHUP,GAME_ID,SEASON,GAME_DATE,HOME_TEAM_NICKNAME,HOME_TEAM_ID,AWAY_TEAM_NICKNAME,AWAY_TEAM_ID


In [4]:
failed_attempts = 0
for _,row in df.iterrows():
    try:
        getSingleGameMetrics(row['GAME_ID'],row['HOME_TEAM_ID'],row['AWAY_TEAM_ID'],row['AWAY_TEAM_NICKNAME'],row['SEASON'],row['GAME_DATE']).to_sql('stg_game_stats', engine, if_exists='append', index=False, method='multi')
    except:
        print(f'failed attempt number {failed_attempts}')
        failed_attempts+= 1
        continue

In [272]:
url = "https://nba-prediction-api.onrender.com/all"
response = requests.get(url)
#historic_games = pd.read_json(response.json(),orient='records')


In [278]:
response.content

b'Internal Server Error'

In [280]:
query = """ select * from fct_wl_features"""
df = pd.read_sql_query(query,engine)
df

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1146 (42S02): target: basketballdb.-.primary: vttablet: Table 'basketballdb.fct_wl_features' doesn't exist (errno 1146) (sqlstate 42S02) (CallerID: 1s12yv1z7l4mbixy53db): Sql: "select * from fct_wl_features", BindVars: {REDACTED}
[SQL:  select * from fct_wl_features]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [207]:
import pandas as pd 
import requests

url = "https://nba-prediction-api.onrender.com/all"
response = requests.get(url)
df = pd.read_json(response.json(),orient='records')
df

,HOME_LAST_GAME_OE,HOME_LAST_GAME_HOME_WIN_PCTG,HOME_NUM_REST_DAYS,HOME_LAST_GAME_AWAY_WIN_PCTG,HOME_LAST_GAME_TOTAL_WIN_PCTG,HOME_LAST_GAME_ROLLING_SCORING_MARGIN,HOME_LAST_GAME_ROLLING_OE,HOME_W,HOME_TEAM_ID,GAME_ID,SEASON,GAME_DATE,AWAY_LAST_GAME_OE,AWAY_LAST_GAME_HOME_WIN_PCTG,AWAY_NUM_REST_DAYS,AWAY_LAST_GAME_AWAY_WIN_PCTG,AWAY_LAST_GAME_TOTAL_WIN_PCTG,AWAY_LAST_GAME_ROLLING_SCORING_MARGIN,AWAY_LAST_GAME_ROLLING_OE,AWAY_TEAM_ID
0,0.535088,NaN,2.0,1.000000,1.000000,15.000000,0.549056,1,1610612737,22000041,2020-21,2020-12-28,0.470588,0.000000,2.0,0.000000,0.000000,-9.500000,0.519910,1610612765
1,0.567797,1.000000,1.0,0.750000,0.800000,7.333333,0.592552,0,1610612737,22000083,2020-21,2021-01-02,0.531532,0.666667,2.0,0.500000,0.600000,-1.666667,0.549664,1610612739
2,0.500000,0.500000,2.0,0.750000,0.666667,3.000000,0.559789,0,1610612737,22000095,2020-21,2021-01-04,0.580357,0.500000,2.0,0.500000,0.500000,-1.333333,0.510969,1610612752
3,0.561404,0.333333,2.0,0.750000,0.571429,2.666667,0.543067,0,1610612737,22000110,2020-21,2021-01-06,0.528000,0.333333,2.0,0.250000,0.285714,-15.666667,0.533390,1610612766
4,0.531532,0.250000,2.0,0.600000,0.444444,-7.000000,0.530978,1,1610612737,22000153,2020-21,2021-01-11,0.536585,0.833333,2.0,0.500000,0.700000,-6.666667,0.559370,1610612755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,0.609756,0.333333,2.0,0.307692,0.320000,-1.666667,0.566357,1,1610612766,22201113,2022-23,2023-03-26,0.584071,0.578947,2.0,0.388889,0.486486,-4.666667,0.580104,1610612742
3536,0.656489,0.351351,3.0,0.325000,0.337662,5.666667,0.613386,0,1610612766,22201148,2022-23,2023-03-31,0.598425,0.526316,2.0,0.421053,0.473684,-4.333333,0.621818,1610612741
3537,0.508197,0.342105,2.0,0.325000,0.333333,-7.000000,0.579533,0,1610612766,22201163,2022-23,2023-04-02,0.623853,0.650000,2.0,0.324324,0.493506,5.666667,0.617840,1610612761
3538,0.571429,0.333333,2.0,0.325000,0.329114,-15.666667,0.578705,0,1610612766,22201176,2022-23,2023-04-04,0.696296,0.650000,2.0,0.342105,0.500000,9.000000,0.655736,1610612761


In [69]:
def test(version):
    query = f"select * from model_object where version = '{version}'"
    df = pd.read_sql_query(query,engine)
    return df.to_json(orient='records')

test('v1')

'[{"model_name":"NBA Home Team Win Loss","version":"v1","create_dt":1699142400000,"model_object_hex":"80049529030000000000008c1e736b6c6561726e2e6c696e6561725f6d6f64656c2e5f6c6f676973746963948c124c6f67697374696352656772657373696f6e9493942981947d94288c0770656e616c7479948c026c32948c046475616c94898c03746f6c94473f1a36e2eb1c432d8c014394473ff00000000000008c0d6669745f696e7465726365707494888c11696e746572636570745f7363616c696e67944b018c0c636c6173735f776569676874944e8c0c72616e646f6d5f7374617465944e8c06736f6c766572948c056c62666773948c086d61785f69746572944b648c0b6d756c74695f636c617373948c046175746f948c07766572626f7365944b008c0a7761726d5f737461727494898c066e5f6a6f6273944e8c086c315f726174696f944e8c0e6e5f66656174757265735f696e5f944b0e8c08636c61737365735f948c156e756d70792e636f72652e6d756c74696172726179948c0c5f7265636f6e7374727563749493948c056e756d7079948c076e6461727261799493944b0085944301629487945294284b014b028594681c8c0564747970659493948c02693894898887945294284b038c013c944e4e4e4affffffff4affffffff4b00

In [300]:
query = "select * from wrk_wl_features where season = '2022-23'"
pd.read_sql_query(query,engine).dropna()

,HOME_LAST_GAME_OE,HOME_LAST_GAME_HOME_WIN_PCTG,HOME_NUM_REST_DAYS,HOME_LAST_GAME_AWAY_WIN_PCTG,HOME_LAST_GAME_TOTAL_WIN_PCTG,HOME_LAST_GAME_ROLLING_SCORING_MARGIN,HOME_LAST_GAME_ROLLING_OE,HOME_W,HOME_TEAM_ID,GAME_ID,SEASON,GAME_DATE,AWAY_LAST_GAME_OE,AWAY_LAST_GAME_HOME_WIN_PCTG,AWAY_NUM_REST_DAYS,AWAY_LAST_GAME_AWAY_WIN_PCTG,AWAY_LAST_GAME_TOTAL_WIN_PCTG,AWAY_LAST_GAME_ROLLING_SCORING_MARGIN,AWAY_LAST_GAME_ROLLING_OE,AWAY_TEAM_ID
3,0.566929,0.666667,3.0,0.600000,0.625000,-8.333333,0.557683,1,1610612737,22200134,2022-23,2022-11-05,0.580357,0.666667,1.0,0.600000,0.625000,9.000000,0.595040,1610612740
4,0.568182,0.750000,2.0,0.600000,0.666667,-4.666667,0.559942,1,1610612737,22200149,2022-23,2022-11-07,0.535088,1.000000,2.0,1.000000,1.000000,17.333333,0.563690,1610612749
5,0.565574,0.800000,2.0,0.600000,0.700000,11.666667,0.566895,0,1610612737,22200162,2022-23,2022-11-09,0.638462,1.000000,2.0,0.571429,0.750000,15.000000,0.595777,1610612762
6,0.590551,0.666667,1.0,0.600000,0.636364,5.333333,0.574769,1,1610612737,22200173,2022-23,2022-11-10,0.514563,0.333333,3.0,0.600000,0.454545,0.000000,0.531692,1610612755
7,0.603604,0.714286,2.0,0.571429,0.642857,4.000000,0.543206,0,1610612737,22200214,2022-23,2022-11-16,0.568000,0.857143,2.0,0.714286,0.785714,10.666667,0.606909,1610612738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,0.609756,0.333333,2.0,0.307692,0.320000,-1.666667,0.566357,1,1610612766,22201113,2022-23,2023-03-26,0.584071,0.578947,2.0,0.388889,0.486486,-4.666667,0.580104,1610612742
1226,0.656489,0.351351,3.0,0.325000,0.337662,5.666667,0.613386,0,1610612766,22201148,2022-23,2023-03-31,0.598425,0.526316,2.0,0.421053,0.473684,-4.333333,0.621818,1610612741
1227,0.508197,0.342105,2.0,0.325000,0.333333,-7.000000,0.579533,0,1610612766,22201163,2022-23,2023-04-02,0.623853,0.650000,2.0,0.324324,0.493506,5.666667,0.617840,1610612761
1228,0.571429,0.333333,2.0,0.325000,0.329114,-15.666667,0.578705,0,1610612766,22201176,2022-23,2023-04-04,0.696296,0.650000,2.0,0.342105,0.500000,9.000000,0.655736,1610612761


In [73]:
import requests 
import pandas as pd
import pickle

url = "https://nba-prediction-api.onrender.com/wl_model?version=%27v1%27"
response = requests.get(url)
data = pd.read_json(response.json())
pickle.loads(bytes.fromhex(data['model_object_hex'][0])).get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [84]:
query = """SELECT gameId as GAME_ID
                     ,homeTeamName as HOME_TEAM_NAME
                     ,awayTeamName as AWAY_TEAM_NAME
                     ,DATE_FORMAT(date(gameDateTimeEST),'%Y-%m-%d') as GAME_DATE_EST
                     ,homeTeamID as HOME_TEAM_ID
                     ,awayTeamID as AWAY_TEAM_ID
            FROM 
                  stg_nba_schedule
            WHERE 
                  postponedStatus = 'A'
              AND (homeTeamId <> 0 and awayTeamID <> 0)
              AND seriesText = '' 
              AND DATE_FORMAT(date(gameDateTimeEST),'%Y-%m-%d') >= curDate() 
            
            ORDER BY
                  gameDateTimeEST ASC
            LIMIT 15      """
df = pd.read_sql_query(query,engine)
df 

,GAME_ID,HOME_TEAM_NAME,AWAY_TEAM_NAME,GAME_DATE_EST,HOME_TEAM_ID,AWAY_TEAM_ID
0,0022300146,Pacers,Spurs,2023-11-06,1610612754,1610612759
1,0022300145,Pistons,Warriors,2023-11-06,1610612765,1610612744
2,0022300148,76ers,Wizards,2023-11-06,1610612755,1610612764
3,0022300147,Magic,Mavericks,2023-11-06,1610612753,1610612742
4,0022300149,Nets,Bucks,2023-11-06,1610612751,1610612749
5,0022300151,Knicks,Clippers,2023-11-06,1610612752,1610612746
6,0022300150,Heat,Lakers,2023-11-06,1610612748,1610612747
7,0022300152,Bulls,Jazz,2023-11-06,1610612741,1610612762
8,0022300155,Thunder,Hawks,2023-11-06,1610612760,1610612737
9,0022300153,Rockets,Kings,2023-11-06,1610612745,1610612758


In [74]:
url = "https://nba-prediction-api.onrender.com/upcoming_games"
response = requests.get(url)
upcoming_games = pd.read_json(response.json(),orient='records')
upcoming_games

,GAME_ID,HOME_TEAM_NAME,AWAY_TEAM_NAME,GAME_DATETIME,HOME_TEAM_ID,AWAY_TEAM_ID
0,22300061,Nuggets,Lakers,2023-10-24T23:30:00Z,1610612743,1610612747
1,22300062,Warriors,Suns,2023-10-25T02:00:00Z,1610612744,1610612756
2,22300064,Pacers,Wizards,2023-10-25T23:00:00Z,1610612754,1610612764
3,22300063,Hornets,Hawks,2023-10-25T23:00:00Z,1610612766,1610612737
4,22300066,Magic,Rockets,2023-10-25T23:00:00Z,1610612753,1610612745
5,22300065,Knicks,Celtics,2023-10-25T23:00:00Z,1610612752,1610612738
6,22300067,Nets,Cavaliers,2023-10-25T23:30:00Z,1610612751,1610612739
7,22300069,Raptors,Timberwolves,2023-10-25T23:30:00Z,1610612761,1610612750
8,22300068,Heat,Pistons,2023-10-25T23:30:00Z,1610612748,1610612765
9,22300070,Bulls,Thunder,2023-10-26T00:00:00Z,1610612741,1610612760


In [336]:
from sklearn import preprocessing


X_test = test_data[['HOME_LAST_GAME_OE', 'HOME_LAST_GAME_HOME_WIN_PCTG',
       'HOME_NUM_REST_DAYS', 'HOME_LAST_GAME_AWAY_WIN_PCTG',
       'HOME_LAST_GAME_TOTAL_WIN_PCTG',
       'HOME_LAST_GAME_ROLLING_SCORING_MARGIN', 'HOME_LAST_GAME_ROLLING_OE',
       'AWAY_LAST_GAME_OE', 'AWAY_LAST_GAME_HOME_WIN_PCTG',
       'AWAY_NUM_REST_DAYS', 'AWAY_LAST_GAME_AWAY_WIN_PCTG',
       'AWAY_LAST_GAME_TOTAL_WIN_PCTG',
       'AWAY_LAST_GAME_ROLLING_SCORING_MARGIN', 'AWAY_LAST_GAME_ROLLING_OE']]
scaler = preprocessing.StandardScaler()
scaler.fit(X_test)
scaled_data = scaler.transform(X_test) 

test_data['PREDICTION'] = model.predict(scaled_data)

test_data['TRUE_POSITIVE_BOOL'] = ((test_data['PREDICTION'] == 1) & (test_data['HOME_W'] == 1))
test_data['TRUE_NEGATIVE_BOOL'] = ((test_data['PREDICTION'] == 0) & (test_data['HOME_W'] == 0))
test_data['FALSE_POSITIVE_BOOL'] = ((test_data['PREDICTION'] == 1) & (test_data['HOME_W'] == 0))
test_data['FALSE_NEGATIVE_BOOL'] = ((test_data['PREDICTION'] == 0) & (test_data['HOME_W'] == 1))
test_data['TRUE_POSITIVE_TOTAL_CNT'] = test_data['TRUE_POSITIVE_BOOL'].cumsum()
test_data['TRUE_NEGATIVE_TOTAL_CNT'] = test_data['TRUE_NEGATIVE_BOOL'].cumsum()
test_data['FALSE_POSITIVE_TOTAL_CNT'] = test_data['FALSE_POSITIVE_BOOL'].cumsum()
test_data['FALSE_NEGATIVE_TOTAL_CNT'] = test_data['FALSE_NEGATIVE_BOOL'].cumsum()

#PRECISION/ACCURACY
test_data['ACCURACY'] = (test_data['TRUE_NEGATIVE_TOTAL_CNT'] + test_data['TRUE_POSITIVE_TOTAL_CNT'])/(test_data['TRUE_NEGATIVE_TOTAL_CNT'] + test_data['TRUE_POSITIVE_TOTAL_CNT'] + test_data['FALSE_NEGATIVE_TOTAL_CNT'] + test_data['FALSE_POSITIVE_TOTAL_CNT']) 
test_data['PRECISION'] = test_data['TRUE_POSITIVE_TOTAL_CNT'] / (test_data['TRUE_POSITIVE_TOTAL_CNT'] + test_data['FALSE_POSITIVE_TOTAL_CNT']) 
test_data

,HOME_LAST_GAME_OE,HOME_LAST_GAME_HOME_WIN_PCTG,HOME_NUM_REST_DAYS,HOME_LAST_GAME_AWAY_WIN_PCTG,HOME_LAST_GAME_TOTAL_WIN_PCTG,HOME_LAST_GAME_ROLLING_SCORING_MARGIN,HOME_LAST_GAME_ROLLING_OE,HOME_W,HOME_TEAM_ID,GAME_ID,...,TRUE_POSITIVE_BOOL,TRUE_NEGATIVE_BOOL,FALSE_POSITIVE_BOOL,FALSE_NEGATIVE_BOOL,TRUE_POSITIVE_TOTAL_CNT,TRUE_NEGATIVE_TOTAL_CNT,FALSE_POSITIVE_TOTAL_CNT,FALSE_NEGATIVE_TOTAL_CNT,ACCURACY,PRECISION
739,0.509615,0.000,2.0,0.000000,0.000000,-5.500000,0.518959,0,1610612755,22200028,...,False,False,True,False,0,0,1,0,0.000000,0.000000
411,0.470588,0.000,3.0,0.000000,0.000000,-10.000000,0.477602,0,1610612747,22200037,...,False,True,False,False,0,1,1,0,0.500000,0.000000
616,0.632000,1.000,3.0,0.000000,0.500000,10.500000,0.590809,1,1610612752,22200046,...,True,False,False,False,1,1,1,0,0.666667,0.500000
165,0.514563,0.000,2.0,0.500000,0.333333,-8.666667,0.522410,1,1610612741,22200047,...,False,False,False,True,1,1,1,1,0.500000,0.500000
329,0.500000,0.000,2.0,0.000000,0.000000,-12.333333,0.531298,1,1610612745,22200048,...,False,False,False,True,1,1,1,2,0.400000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.603448,0.525,2.0,0.439024,0.481481,-9.333333,0.555028,1,1610612741,22201223,...,True,False,False,False,477,235,257,210,0.603902,0.649864
983,0.557377,0.575,3.0,0.390244,0.481481,-1.666667,0.566125,1,1610612760,22201226,...,False,False,False,True,477,235,257,211,0.603390,0.649864
1024,0.554545,0.650,2.0,0.341463,0.493827,-1.000000,0.564518,1,1610612761,22201221,...,False,False,False,True,477,235,257,212,0.602879,0.649864
286,0.557252,0.825,1.0,0.463415,0.641975,-9.666667,0.576066,1,1610612743,22201227,...,False,False,False,True,477,235,257,213,0.602369,0.649864


In [1]:
import pandas as pd

students = {"name" : ["Tim Voss", "Nicole Johnson", "Elsa Williams", "John James", "Catherine Jones"], "age" : [19, 20, 21, 20, 23], "favorite_color" : ["red", "yellow", "green", "blue", "green"], "grade" : [91, 95, 82, 75, 93]}

students_df = pd.DataFrame(students)


In [17]:
students_df[((students_df['favorite_color'] == 'red') | (students_df['favorite_color'] == 'green')) & (students_df['grade'] > 90) ]

,name,age,favorite_color,grade
0,Tim Voss,19,red,91
4,Catherine Jones,23,green,93
